## (주)다유푸드 제품 2개 리뷰 크롤링 

 ### 당플랜 소고기 볶음밥, 당플랜 돼지고기 볶음밥

In [15]:
import pandas as pd

# 파일 경로
file_path = r'C:\Users\USER\Desktop\5_6\core_pj\data\crawlingOrNogada\hyundaiGreenFoodAllReview.xlsx'

# 엑셀 파일 로드
df = pd.read_excel(file_path)

# 모든 리뷰 데이터를 하나의 텍스트 파일로 저장
output_file_path = r'C:\Users\USER\Desktop\5_6\core_pj\data\all_reviews.txt'
with open(output_file_path, 'w', encoding='utf-8') as file:
    for index, row in df.iterrows():
        review_text = str(row['리뷰']) if not pd.isna(row['리뷰']) else ""
        file.write(review_text + '\n')

print(f"모든 리뷰가 {output_file_path}에 저장되었습니다.")


모든 리뷰가 C:\Users\USER\Desktop\5_6\core_pj\data\all_reviews.txt에 저장되었습니다.


In [26]:
from selenium import webdriver as wb
from selenium.webdriver.common.by import By
from selenium.webdriver.chrome.service import Service
from webdriver_manager.chrome import ChromeDriverManager
import pandas as pd
import time

# 드라이버 설정
driver = wb.Chrome(service=Service(ChromeDriverManager().install()))

# 사이트 열기
url = "https://www.wellife.co.kr/products/view/G2001000864#enp_mbris"
# 다유푸드 제품 해당 url만 바꿔서 넣어주면 리뷰 긁어올 수 있음
driver.get(url)
time.sleep(2)  # 페이지 로딩 대기

# 광고 페이지 닫기
ad_close_selector = '#is2popup1 > div.is2_popup-content > div > div.is2_popup-buttons > button.is2_popup-button.is2_popup-button-today'
try:
    ad_close_button = driver.find_element(By.CSS_SELECTOR, ad_close_selector)
    ad_close_button.click()
    time.sleep(1)  # 광고 닫기 대기
except Exception as e:
    print("광고 페이지가 발견되지 않음:", e)

# 후기 탭 클릭
review_tab_selector = '#saleson > section > div.item_tab.widowScrolled > div > ul > li:nth-child(3) > a > span.txt'
review_tab = driver.find_element(By.CSS_SELECTOR, review_tab_selector)

# 스크롤 이동 후 클릭
driver.execute_script("arguments[0].scrollIntoView();", review_tab)
time.sleep(1)
driver.execute_script("arguments[0].click();", review_tab)
time.sleep(2)  # 페이지 로딩 대기

# 상품명 가져오기
product_name_selector = '#saleson > section > div.view_top.container > div > div.col-12.col-lg-6.view_info > div.tit_area > p.title'
product_name = driver.find_element(By.CSS_SELECTOR, product_name_selector).text

reviews = []
page = 1

while True:
    try:
        print(f"Scraping page {page}...")

        # 리뷰 텍스트 가져오기
        review_texts = driver.find_elements(By.CSS_SELECTOR, '#review-list > li > div > div > div.review_origin > div.txt_area > div')
        for review in review_texts:
            reviews.append(review.text)

        # 다음 페이지 버튼 클릭
        next_button = driver.find_element(By.CSS_SELECTOR, '#review-list > ul > li.page-item.next > a')
        if next_button.is_displayed():
            driver.execute_script("arguments[0].scrollIntoView();", next_button)
            time.sleep(1)
            driver.execute_script("arguments[0].click();", next_button)
            time.sleep(2)  # 페이지 로딩 대기
            page += 1
        else:
            break
    except Exception as e:
        print(f"Exception occurred: {e}")
        break

# 데이터프레임 생성
df = pd.DataFrame(reviews, columns=["Review"])
df.insert(0, "Product Name", product_name)

# 크롤링 결과 출력
print(df)

# 크롬 드라이버 종료
# driver.quit()


Scraping page 1...
Scraping page 2...
Scraping page 3...
Scraping page 4...
Scraping page 5...
Scraping page 6...
Scraping page 7...
Scraping page 8...
Scraping page 9...
Scraping page 10...
Scraping page 11...
Scraping page 12...
Scraping page 13...
Scraping page 14...
Scraping page 15...
Scraping page 16...
Scraping page 17...
Scraping page 18...
Scraping page 19...
Scraping page 20...
Scraping page 21...
Scraping page 22...
Scraping page 23...
Scraping page 24...
Scraping page 25...
Scraping page 26...
Scraping page 27...
Scraping page 28...
Scraping page 29...
Scraping page 30...
Scraping page 31...
Scraping page 32...
Scraping page 33...
Scraping page 34...
Scraping page 35...
Scraping page 36...
Scraping page 37...
Scraping page 38...
Scraping page 39...
Scraping page 40...
Scraping page 41...
Scraping page 42...
Exception occurred: Message: no such element: Unable to locate element: {"method":"css selector","selector":"#review-list > ul > li.page-item.next > a"}
  (Session info:

이렇게 수집해서 df를 엑셀로 저장해놓고 ' (6ea)' 부분 삭제

## 나중에 리뷰 분류를 직접 해보고 싶다면 이런식으로 할 수 있음

In [ ]:
import pandas as pd
from sklearn.model_selection import train_test_split
from sklearn.feature_extraction.text import TfidfVectorizer
from sklearn.linear_model import LogisticRegression
from sklearn.metrics import classification_report

# 파일 경로
file_path = r'C:\Users\USER\Desktop\5_6\core_pj\data\crawlingOrNogada\hyundaiGreenFoodAllReview.xlsx'

# 엑셀 파일 로드
df = pd.read_excel(file_path)

# 예시로 감성 레이블을 추가 (데이터에 실제 레이블이 있어야 함)
# 실제 데이터에는 수동으로 레이블링하거나 다른 방법으로 레이블링해야 합니다.
# 여기서는 임의로 0 (부정), 1 (긍정) 레이블을 추가합니다.
# df['sentiment'] = [1, 0, 1, 0, 1]  # 예시 레이블, 실제 데이터에 맞게 수정 필요

# 데이터와 레이블 분리
X = df['리뷰']
# y = df['sentiment']

# 데이터셋 분할
X_train, X_test, y_train, y_test = train_test_split(X, y, test_size=0.2, random_state=42)

# 텍스트 벡터화 (TF-IDF)
vectorizer = TfidfVectorizer()
X_train_vec = vectorizer.fit_transform(X_train)
X_test_vec = vectorizer.transform(X_test)

# 로지스틱 회귀 모델 학습
model = LogisticRegression()
model.fit(X_train_vec, y_train)

# 예측
y_pred = model.predict(X_test_vec)

# 결과 출력
print(classification_report(y_test, y_pred))

# 감성 점수를 -2에서 2까지의 범주로 변환
def sentiment_to_category(score):
    if score >= 1.5:
        return 2
    elif score >= 0.5:
        return 1
    elif score >= -0.5:
        return 0
    elif score >= -1.5:
        return -1
    else:
        return -2

# 예측 점수를 범주로 변환하여 데이터프레임에 추가
df['predicted_sentiment'] = model.predict(vectorizer.transform(df['리뷰']))
df['sentiment_category'] = df['predicted_sentiment'].apply(sentiment_to_category)

# 결과 엑셀 파일로 저장
output_file_path = r'C:\Users\USER\Desktop\5_6\core_pj\data\crawlingOrNogada\hyundaiGreenFoodSentimentAnalysis.xlsx'
df.to_excel(output_file_path, index=False)

print(f"결과가 {output_file_path}에 저장되었습니다.")
